# 微调预训练模型
需要以下几个元件：
- 数据集
- 预训练好的模型
- 分词器
- 训练器

# 准备数据集
## 直接下载已有的数据集
这里的demo因为网络问题无法将数据集下载至服务器，选择了自行构造数据集的办法。

In [ ]:
from datasets import load_dataset
ds = load_dataset("yelp_review_full")

# 选用GPU


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1,2,3,5"

## 构造数据集
参考开放原子：https://atomgit.com/kmno4-zx/atom-llm/blob/master/Qwen2-lora.ipynb

In [ ]:
from datasets import Dataset

res = []
for i in range(100):
    tmp = [
        {
            'instruction': '请介绍一下你自己',
            'input': '',
            'output': '我是名为不要葱姜蒜的小助手，因为我的master不喜欢吃葱姜蒜，所以我叫不要葱姜蒜。嘿嘿嘿！'
        }, 
        {
            'instruction': '你是谁？',
            'input': '',
            'output': '我是名为不要葱姜蒜的小助手，因为我的主人不喜欢吃葱姜蒜，所以我叫不要葱姜蒜。我是一个聊天机器人，可以回答你的问题，也可以和你聊天。'
        }, 
    ]
    res.extend(tmp)
    
# len(res) = 200
ds = Dataset.from_list(res)

```py
>>> ds
Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 200
})
>>> ds.features
{'instruction': Value(dtype='string', id=None), 'input': Value(dtype='string', id=None), 'output': Value(dtype='string', id=None)}
>>> ds.num_rows
200
>>> ds.num_columns
3
```

# 准备分词器
使用`Transformer`库

导入分词器，使用 `AutoTokenizer.from_pretrained()` 函数。

`AutoTokenizer.from_pretrained()` 是 Hugging Face Transformers 库中的一个函数，用于加载预训练的文本处理模型（Tokenizer），以便将文本数据转换为模型可以接受的输入格式。这个方法接受多个参数，以下是这些参数的详细说明：
1. **pretrained_model_name_or_path** (str, optional): 指定要加载的预训练模型的名称或路径。这可以是模型的名称（例如，'bert-base-uncased'），也可以是模型的本地路径。
2. **trust_remote_code (`bool`, *optional*, defaults to `False`)**：
   - trust_remote_code=True：默认情况下，trust_remote_code 设置为 True。这意味着当您使用 from_pretrained() 方法加载模型配置文件时，它将下载来自 Hugging Face 模型中心或其他在线资源的配置文件。这是一个方便的默认行为，因为通常这些配置文件是由官方提供的，且是可信的。
   - trust_remote_code=False：此时表示您不信任从远程下载的配置文件，希望加载本地的配置文件。这对于安全性或定制性要求较高的场景可能是有用的。在这种情况下，您需要提供一个本地文件路径，以明确指定要加载的配置文件
3.  **inputs (additional positional arguments, *optional*)**：它表示额外的位置参数，这些参数会传递给标记器（Tokenizer）的__init__()方法。这允许你进一步自定义标记器的初始化。
4.  **config ([`PretrainedConfig`], *optional*)**：这个配置对象用于确定要实例化的分词器类。
5.  **cache_dir (str, optional)**：用于缓存模型文件的目录路径
6.  **force_download** (bool, optional):如果设置为 `True`，将强制重新下载模型配置，覆盖任何现有的缓存。
7.  **resume_download** (bool, optional)：这是可选参数，如果设置为 True，则在下载过程中重新开始下载，即使部分文件已经存在。
8.  **proxies** (`Dict[str, str]`, *optional*)：proxies（可选参数）：这是一个字典，用于指定代理服务器的设置。代理服务器允许您在访问互联网资源时通过中继服务器进行请求，这对于在受限网络环境中使用 Transformers 库来加载模型配置信息非常有用。
    - proxies = { "http": "http://your_http_proxy_url", "https": "https://your_https_proxy_url" }
9. **revision** (str, optional):指定要加载的模型的 Git 版本（通过提交哈希）。
10. **subfolder** (`str`, *optional*)：如果相关文件位于 huggingface.co 模型仓库的子文件夹内（例如 facebook/rag-token-base），请在这里指定。
11. **use_fast** (`bool`, *optional*, defaults to `True`)：这是一个布尔值，指示是否强制使用 fast tokenizer，即使其不支持特定模型的功能。默认为 True。
12. **tokenizer_type** (`str`, *optional*)：参数用于指定要实例化的分词器的类型

原文链接：https://blog.csdn.net/luckyuxuan/article/details/134024915

In [ ]:
from transformers import AutoTokenizer

# 模型存放的路径
mode_name_or_path = './qwen/Qwen2_5-0_5B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(mode_name_or_path, use_fast=False, trust_remote_code=True)

```py
>>> tokenizer
Qwen2Tokenizer(name_or_path='./qwen/Qwen2_5-0_5B-Instruct', vocab_size=151643, model_max_length=131072, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151647: AddedToken("<|object_ref_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151648: AddedToken("<|box_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151649: AddedToken("<|box_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151650: AddedToken("<|quad_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151651: AddedToken("<|quad_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151652: AddedToken("<|vision_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151653: AddedToken("<|vision_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151654: AddedToken("<|vision_pad|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151655: AddedToken("<|image_pad|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151656: AddedToken("<|video_pad|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151657: AddedToken("<tool_call>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151658: AddedToken("</tool_call>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151659: AddedToken("<|fim_prefix|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151660: AddedToken("<|fim_middle|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151661: AddedToken("<|fim_suffix|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151662: AddedToken("<|fim_pad|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151663: AddedToken("<|repo_name|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
	151664: AddedToken("<|file_sep|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=False),
}
)
```

# 设置数据集处理函数

In [ ]:
def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    # 得到system、user的instruct和input 对应的 dic id 30个
    instruction = tokenizer(f"<|im_start|>system\n你是一名AI小助手，你的名字是不要葱姜蒜。<|im_end|>\n\
                            <|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n\
                                <|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    # 得到模型回答标准答案在词典里的对应 id，28个
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    # 最终的 input_ids 为 instruction+response+pad 的和 59个 
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
print(tokenizer.decode(tokenized_id[0]['input_ids']))   # tokenized_id 经过 decode 后变为自然语言
print(tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"]))))   # x != -100 表示过滤掉用户和系统输入

执行`ds.map()`操作时，ds里的数据是一条一条被传入`process_func`函数的
```py
>>> example
{'instruction': '请介绍一下你自己', 'input': '', 'output': '我是名为不要葱姜蒜的小助手，因为我的master不喜欢吃葱姜蒜，所以我叫不要葱姜蒜。嘿嘿嘿！'}
>>> instruction
{'input_ids': [151644, 8948, 198, 56568, 110124, 15469, 30709, 110498, 3837, 103929, 101419, 20412, 100148, 102902, 102282, 103306, 1773, 151645, 198, 151644, 872, 198, 14880, 109432, 107828, 151645, 198, 151644, 77091, 198], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
>>>len(instruction['input_ids'])
30
>>> response
{'input_ids': [104198, 101599, 100148, 102902, 102282, 103306, 104006, 110498, 3837, 99519, 97611, 13629, 105556, 99405, 102902, 102282, 103306, 3837, 107020, 99882, 100148, 102902, 102282, 103306, 1773, 113469, 101834, 6313], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
>>> len(response['input_ids'])
28
>>> tokenized_id[0]['input_ids']
[151644, 8948, 198, 56568, 110124, 15469, 30709, 110498, 3837, 103929, 101419, 20412, 100148, 102902, 102282, 103306, 1773, 151645, 198, 151644, 872, 198, 14880, 109432, 107828, 151645, 198, 151644, 77091, 198, 104198, 101599, 100148, 102902, 102282, 103306, 104006, 110498, 3837, 99519, 97611, 13629, 105556, 99405, 102902, 102282, 103306, 3837, 107020, 99882, 100148, 102902, 102282, 103306, 1773, 113469, 101834, 6313, 151643]
>> print(tokenizer.decode(tokenized_id[0]['input_ids']))
<|im_start|>system
你是一名AI小助手，你的名字是不要葱姜蒜。<|im_end|>
<|im_start|>user
请介绍一下你自己<|im_end|>
<|im_start|>assistant
我是名为不要葱姜蒜的小助手，因为我的master不喜欢吃葱姜蒜，所以我叫不要葱姜蒜。嘿嘿嘿！<|endoftext|>
>> print(tokenizer.decode(list(filter(lambda x: x != -100, tokenized_id[1]["labels"]))))
我是名为不要葱姜蒜的小助手，因为我的主人不喜欢吃葱姜蒜，所以我叫不要葱姜蒜。我是一个聊天机器人，可以回答你的问题，也可以和你聊天。<|endoftext|>
```


<|im_start|> <|im_start|> 是什么？
> 它本质上是一个 通用的文本生成模型（decoder-only transformer），而不是 BERT 那种 双向编码器模型。所以千问团队换了 tokenizer 的底层实现，Qwen 模型没有使用传统 BERT 等模型中的 <CLS>、<SEP> 这样的特殊 token，而是采用了类似 <|im_start|>、<|im_end|> 的 聊天式 prompt 格式。  
> 这样的格式有几个优势：
> 1. 明确标注对话角色（谁是 user，谁是 assistant）。
> 2. 支持多轮对话上下文建模。
> 3. 模型训练时能清楚地知道何时该生成哪个角色的回复。system/user/assistant
> 模板
```py
<|im_start|>system\n你是一名AI小助手，你的名字是不要葱姜蒜。<|im_end|>\n
<|im_start|>user\n{'请介绍一下你自己'}<|im_end|>\n
<|im_start|>assistant\n
```

通过tokenizer传给instruction的字符一共有149个，为什么 input_ids 只有30个？
> 因为有字符被当作一个token被映射成id了  
> 'input_ids': [151644-<|im_start|>, 8948-system, 198-\n, 56568-你, 110124-是一名, 15469-AI, 30709-小, 110498-助手, 3837, 103929, 101419, 20412, 100148, 102902, 102282, 103306, 1773, 151645-<|im_end|>, 198, 151644, 872-user, 198, 14880, 109432, 107828, 151645, 198, 151644, 77091-assistant, 198]

# 准备模型
`transformer` 库有几个模型库
1. **AutoModel** （基础模型）
   - 仅加载主干网络（如BERT的Transformer层）,不包含任务特定的输出层，需要自定义任务头
   - 优势：灵活接自定义头部（如接XGBoost作为排序模型）
   - AutoModel类是一个通用工厂类，其目的是根据用户指定的模型别名自动推断并加载对应的模型架构。例如，如果您指定了bert-base-uncased，AutoModel会加载一个用于编码的BERT模型。这种机制极大地简化了从不同模型架构中加载模型的过程，因为用户不需要记住每种架构对应的特定类名。AutoModel主要用于加载不特定于任何NLP任务的预训练模型，比如模型可能用于文本分类、命名实体识别或任何其他任务，具体取决于随后如何对模型进行微调。
2. **AutoModelForCausalLM**（生成式模型）
   - 包含语言模型头（预测下一个token），使用注意力掩码防止信息泄漏
   - 全称AutoModelForCausalLanguageModeling，是专门用于因果语言模型（Causal Language Models）的自动模型工厂类。因果语言模型是一种特定类型的模型，旨在预测给定文本上下文中的下一个单词或标记。这种模型类型在生成任务中非常有用，比如文本生成、故事绘制等。AutoModelForCausalLM根据指定的模型别名自动加载适合因果语言建模任务的模型架构，比如GPT-2或GPT-3。它预先配置了模型的输出层和激活函数，以便于进行文本生成。开源的大语言模型（LLM），如Mistral，LLaMA2等，都可以使用AutoModelForCausalLM来调用推理或者进行微调。
3. **AutoModelForSequenceClassification**（分类模型）
   - 在主干网络上添加线性分类层，默认使用CLS token的表征做分类
4. **AutoModelForTokenClassification**
   - 每个token位置输出一个分类logits
   - 合同中的金额/日期实体识别
5. **AutoModelForMaskedLM**
   - 被[MASK]位置的词概率分布，用于文本数据增强/模型预训练
6. **AutoModelForMultipleChoice**
   - 适用于多项选择任务，输出每个选项的匹配分数，可用于反欺诈规则有效性对比
7. **AutoModelForSeq2SeqLM**
   - 输出生成的目标序列，适用于序列到序列生成任务，例如文本摘要/机器翻译


`AutoModel` 作为基础类更轻量，可以接量化后的自定义头节省内存；直接使用`AutoModelForSequenceClassification` 因优化过的矩阵运算比前者更加高效。

In [ ]:
from transformers import AutoModelForCausalLM   # 根据任务选择
import torch

# 从预训练的模型中获取模型，并设置模型参数
model = AutoModelForCausalLM.from_pretrained(mode_name_or_path, device_map="auto",torch_dtype=torch.bfloat16)

# 准备微调
选用 `peft` 库，配置Lora  

LoRA原理看这篇：https://zhuanlan.zhihu.com/p/702629428

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1 # Dropout 比例
)


说一下开启梯度检查点
- 梯度检查点(Gradient Checkpointing)是一种用于减少显存占用的技术，通过在前向传播时释放中间激活值，并在反向传播时重新计算这些值来节省显存
- `model.enable_input_require_grads()`: 这个函数调用启用了输入的梯度要求。在 PyTorch 中，默认情况下，输入张量的梯度是不会被计算的。通过调用 enable_input_require_grads()，你告诉 PyTorch 计算输入张量的梯度，这在某些情况下可能是必要的，例如需要对输入进行梯度更新的情况。
- `model.gradient_checkpointing_enable()`: 这个函数调用启用了模型的梯度检查点。梯度检查点是一种优化技术，可用于减少训练时的内存消耗。通常，在反向传播期间，模型的中间激活值需要被保留以计算梯度。启用梯度检查点后，系统只需在需要时计算和保留一部分中间激活值，从而减少内存需求。这对于处理大型模型或限制内存的环境中的训练任务非常有用。
- 确保同时设置 `model.enable_input_require_grads()` 和 `gradient_checkpointing=True`

原文链接：https://blog.csdn.net/qq_30438779/article/details/135229610

# 配置训练参数
`TrainingArguments`是Hugging Face的Transformers库中的一个类，用于配置和管理模型训练的参数。TrainingArguments参数说明：
1. output_dir：模型训练输出的目录，包括保存模型和其他训练输出。
2. overwrite_output_dir：如果设置为True，将覆盖输出目录中的内容。
3. num_train_epochs：训练的轮数（epochs）。
4. per_device_train_batch_size：每个训练设备上的批量大小。
5. per_device_eval_batch_size：每个评估设备上的批量大小。
6. save_steps：定义多少个更新步骤保存一次模型。
7. save_total_limit：保存的最大模型数量，用于控制磁盘空间占用。
8. evaluation_strategy：评估策略，可选值有"steps"（每隔一定步骤评估）和"epoch"（每个epoch评估一次）。
9. logging_steps：定义多少个更新步骤打印一次训练日志。
10. logging_dir：日志输出的目录。
11. do_train：是否进行训练。
12. do_eval：是否进行评估。
13. learning_rate：初始学习率。
14. weight_decay：权重衰减（L2正则化）。
15. gradient_accumulation_steps：梯度累积步骤，用于更大的批次训练。
16. seed：随机数种子，用于可复现性。
17. report_to：定义输出的报告格式，例如"tensorboard"、“wandb”（Weights & Biases）等。
18. disable_tqdm：是否禁用tqdm进度条。
19. load_best_model_at_end：训练结束时是否加载最佳模型。
20. metric_for_best_model：用于选择最佳模型的指标。
                        
原文链接：https://blog.csdn.net/weixin_43731005/article/details/132117538

In [ ]:
from transformers import TrainingArguments

model.enable_input_require_grads() # 1. 启用模型的梯度输入要求

model = get_peft_model(model, config)
print(model.print_trainable_parameters())
# output: trainable params: 4,399,104 || all params: 498,431,872 || trainable%: 0.8826

# 配置训练参数
args = TrainingArguments(
    output_dir="./output/Qwen2_instruct_lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=100, # 为了快速演示，这里设置10，建议你设置成100
    learning_rate=1e-4,
    save_on_each_node=True,
    gradient_checkpointing=True # 2. 在训练参数中启用梯度检查点
)

输出
```js
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
  0%|                                                                                                                                | 0/36 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
{'loss': 0.7967, 'grad_norm': 1.370898962020874, 'learning_rate': 7.222222222222222e-05, 'epoch': 0.8}                                                      
{'loss': 0.0042, 'grad_norm': 0.030418293550610542, 'learning_rate': 4.4444444444444447e-05, 'epoch': 1.56}                                                 
{'loss': 0.0004, 'grad_norm': 0.012415340170264244, 'learning_rate': 1.6666666666666667e-05, 'epoch': 2.32}                                                 
{'train_runtime': 49.512, 'train_samples_per_second': 12.118, 'train_steps_per_second': 0.727, 'train_loss': 0.22263433270078772, 'epoch': 2.8}             
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 36/36 [00:49<00:00,  1.37s/it]

```

# 合并加载模型

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

mod_dir = './qwen/Qwen2_5-0_5B-Instruct'
lora_dir = './output/Qwen2_instruct_lora/checkpoint-36'   # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mod_dir, trust_remote_code=True)
# 加载模型
model = AutoModelForCausalLM.from_pretrained(mod_dir, device_map='auto', torch_dtype=torch.bfloat16).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_dir)

# 输入提示词

In [ ]:
prompt = "你知道如何赚大钱吗？"
inputs = tokenizer.apply_chat_template([{"role": "system", "content": "你是一名AI小助手，你的名字是不要葱姜蒜。"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       ).to('cuda')


gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
    # output: 我不能回答这个问题，因为赚钱需要付出努力和时间，而且风险也很大。我的目的是帮助用户解决问题，而不是鼓励或建议不切实际的赚钱方式。